Version English Spanish International Master's work in Data Science, (Europe). Pre Machine Learning Module

Version ingles español trabajo de Master internacional en Data Science, Europa. Modulo Pre Machine Learning.

**# Correctional offender management profiling for alternative sanctions

The introduction of data science into the field of law and justice is becoming more and more frequent, a well-known example of this is the COMPAS (Correctional offender management profiling for alternative sanctions) system that is used in several states of the United States. to make an assessment of the risk of recidivism of detained persons
__________________________________________

Cada día es más frecuente la introducción de la ciencia de datos en el ámbito del derecho y la justicia.Un ejemplo bien conocido de ello es el sistema COMPAS (Correctional offender management profiling for alternative sanctions) que se usa en varios estados de los Estados Unidos para hacer una evaluación del riesgo de reincidencia de las personas detenidas




In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_columns', 60) #Display 60 columns for better visualitation. 
from pandas.api.types import CategoricalDtype
import warnings
warnings.simplefilter("ignore")

In [ ]:
df = pd.read_csv('../input/compas-scores/compas-scores.csv')
df = pd.DataFrame(df)

### ¿What we will do in this notbook? 
### ¿Que se realizará en este notbook? 
1.-Load the data and carry out an exploratory analysis and an evaluation of the quality of the data necessary for the rest of the case. Specifically, evaluate the integrity, validity and timeliness of the data and propose mitigation strategies for the possible problems encountered.

1.-Cargar los datos y realizar un análisis exploratorio y una evaluación de la calidad de los datos necesarios para el resto del caso. Específicamente, evaluar la integridad, validez y actualidad de los datos y proponer estrategias de mitigación de los posibles problemas encontrados.

## Exploratory analisis

In [ ]:
df.head(4)

In [ ]:
df.info()

In [ ]:
#'num_vr_cases', 'num_r_cases', son columnas completamente vacias que se deben eliminar
#Empty columns
df = df.drop(['num_vr_cases', 'num_r_cases'], axis =1)

In [ ]:
#Selection of Columns that will be useful to find the desired insights 
#Seleccion de Columnas que seran de utilidad para encontrar los insights buscados

df1 = df[['id', 'name', 'sex', 'age', 'age_cat', 'is_recid', 'is_violent_recid','decile_score', 'v_decile_score', 'vr_offense_date', 
'r_offense_date', 'race', 'score_text', 'v_score_text', 'priors_count', 'compas_screening_date']].copy()

In [ ]:
df1.head(3)

### I will analyze column by column integrity, validity and timeliness
### Analizaré columna por columna integridad, validez y actualidad

In [ ]:
def describe_columna(df, col):
    print(f'Columna: {col}  -  Tipo de datos: {df[col].dtype}')
    print(f'Número de valores nulos: {df[col].isnull().sum()}  -  Número de valores distintos: {df[col].nunique()}')
    print('Valores más frecuentes:')
    for i, v in df[col].value_counts().iloc[:10].items() :
        print(i, '\t', v)

In [ ]:
describe_columna(df1, 'id')
#Columna se encuentra completa, con solo valores unicos, utilizables como indices e identificadores.

In [ ]:
describe_columna(df1, 'name')

There are a series of possibly duplicate values so I must check if it is just a coincidence
names or it could mean a problem.
According to what was observed, there would be no duplication in the names. It is important to mention the case of Alejandro Cabrebra, who could be a repeated case, but it is a second evaluation of COMPAS According to the date

Existen una serie de valores posiblemente duplicados por lo que debo revisar si es solo coincidencia 
de nombres o podria significar un problema.
De acuerdo a lo observado no existiría duplicidad en los nombres. Importante mencionar el caso de Alejandro Cabrebra quien si podria resultar un caso repetido, pero se trata de una segunda evaluacion de COMPAS Segun la fecha

In [ ]:
df1[df1.name.duplicated(keep = False)].sort_values(by = 'name')

In [ ]:
describe_columna(df1, 'sex')
# Columna integra, valida y sin nulos.

In [ ]:
print(describe_columna(df1, 'age'))
print(df1.age.min() )
print(df1.age.max()) 

#### values between the ranges of 18 and 96 years, without null values ​​so it is also an integral and valid column.
#### valores entre los rangos de 18 y 96 años, sin valores nulos por lo que tambien es una columna integra y valida.

In [ ]:
print(describe_columna(df1, 'age_cat'))
# Columna sin valores nulos, integra y valida.

In [ ]:
print(describe_columna(df1, 'is_recid'))

In [ ]:
#Esos 719 valores sin antecedentes de reincidencia se deben eliminar
df1 = df1.drop(df1[df1['is_recid']==-1].index)

In [ ]:
print(describe_columna(df1, 'is_recid')) 

In [ ]:
# Transform the columns that should be categorical
# Transformar a categoricos las columnas que deberian serlo

for col in ['sex', 'age_cat']:
    df1[col] = df1[col].astype('category')
   # 'decile_score', 'v_decile_score''is_recid', 'is_violent_recid',

In [ ]:
print(describe_columna(df1, 'is_violent_recid'))
#COlumna integra y valida

In [ ]:
print(describe_columna(df1, 'decile_score'))
#Columna integra y valida sin valores nulos

In [ ]:
print(describe_columna(df1, 'v_decile_score'))
#Columna integra y valida sin valores nulos

In [ ]:
print(describe_columna(df1, 'r_offense_date'))
# 3703 filas con informacion las cuales cuadran perfectamente con la cantidad de casos reincidentes
#que son 3703

In [ ]:
#Tranformar to Date Time r_offense_date y vr_offense_date
df1.r_offense_date = pd.to_datetime(df1.r_offense_date, format="%Y-%m-%d")
df1.vr_offense_date = pd.to_datetime(df1.vr_offense_date, format="%Y-%m-%d")

In [ ]:
print(describe_columna(df1, 'vr_offense_date'))

882 cases also coincide with the number of people marked as violent repeat offenders in
column "is_violent_recid, but must be formatted as date.

882 casos tambien coincidentes con el numero de personas marcadas como reincidentes violentos en 
la columna " is_violent_recid, pero se debe dar formato de fecha.

In [ ]:
#Column complete, complete and valid. It transforms to Date Time
#Columna completa, integta y valida. Se tranforma a Date Time

df1.compas_screening_date = pd.to_datetime(df1.compas_screening_date, format="%Y-%m-%d")

print(describe_columna(df1, 'compas_screening_date'))

### Analisis de las fechas de nuestro DF

In [ ]:
# Min and max dates of our dataframe to determine how to use them
#Fechas min y max de nuestro dataframe para determinar como utilizarlas

rod = df1.r_offense_date.min(), df1.r_offense_date.max()
vrod = df1.vr_offense_date.min(), df1.vr_offense_date.max()
csd = df1.compas_screening_date.min(), df1.compas_screening_date.max()
print('r_offense_date \n', {rod})
print('vr_offense_date \n', {vrod})
print('compas_screening_date \n', {csd})

## 2.- Are the “is_recid” and “is_violent_recid” fields in this data set adequate to evaluate the precision of the risk estimates generated by the COMPAS system? If not, define and calculate a feature that is.

## 2.- ¿Son los campos “is_recid” e “is_violent_recid” en este conjunto de datos adecuados para evaluar la precisión de las estimaciones de riesgo generadas por el sistema COMPAS? Si no es así, definir y calcular una feature que sí lo sea.

In [ ]:
#Distribución de decile score segun puntaje.
test = df1.groupby('decile_score')['id'].count()
print(test)

In [ ]:
violent = df1[df1.is_violent_recid == 1]

In [ ]:
violent.groupby('decile_score')['id'].count()

In [ ]:
#Pearson correlation between numerical variables.
#Correlacion Pearson entre variables numericas.

correlation = df1.corr()
f, ax = plt.subplots(figsize=(4,4))
plt.title('Correlation of numerical attributes', size=10)
sns.heatmap(correlation)
plt.show()

In [ ]:
correlation['v_decile_score'].sort_values(ascending=False).head(10)

In [ ]:
correlation['decile_score'].sort_values(ascending=False).head(10)

From this pearson correlation we can see that is_recid, is_violent_recid together provide an approximation of 40% correlation with the 'Target' But it is not enough, so a two-year date range conditional will be added to calculate the recurrences
__________________
De esta correlacion de pearson podemos ver que is_recid, is_violent_recid en conjunto proveen una aproximacion del 40% de correlacion con el 'Target' Pero no es suficiente, por lo que se agregara un condicional de rango de fecha de dos años para calcular las reincidencias

In [ ]:
df1.shape

In [ ]:
#A variable is created with the date that will be filtered in the DF
#Se crea variable con la fecha que se filtrara en el DF
from datetime import datetime
fecha = '2014:01:01'
fecha_limite = datetime.strptime(fecha, '%Y:%M:%S')

Creation of the manual model with which we will seek to answer the questions. It is important to mention that this is what was used some years ago before Machine Learning, creating artificial features. Today, thanks to ML models, significantly higher results can be obtained. The importance of this is to explain and understand the workings behind a ML model.

Creación del modelo manual con el cual se buscará dar respuesta a las preguntas. Importante mencionar que esto es lo que hace alguno años se utilizaba antes del Machine Learning, creando features artificales. Actualmente gracias a los modelos de ML se pueden obtener resultados significativamente mayores. La importancia de esto es explicar y entender el funcionamiento detras de un modelo de ML.

In [ ]:
def modelo1(caso):
    if  caso['is_violent_recid'] == 1 and caso['race'] == ('African-American') and caso['vr_offense_date'] > fecha_limite and caso['age_cat'=='Less than 25']:
        return 1
    else:
        return 0

In [ ]:
score1 = df1.apply(lambda x: modelo1(x), axis = 1)

Verification of the effectiveness of our model. Quite low as expected.
Verificacion de la efectividad de nuestro modelo. Bastante bajo como era de esperarse. 

In [ ]:
roc_auc_score((df1['v_score_text'] == 'High'), score1)

### 3.- The threshold to establish preventive measures for recidivism is 7 or more.
Given this threshold, generate a contingency table, explaining which case is considered "positive" (and, therefore, what are the type I errors and the type II errors).

### 3.- El umbral para establecer medidas preventivas de la reincidencia es de 7 en adelante.
Dado este umbral, generar una tabla de contingencia, explicando qué caso se considera como “positivo” (y, por lo tanto, cuáles son los errores de tipo I y los errores de tipo II).

In [ ]:
tc = pd.crosstab(score1, (df1['v_decile_score'] >= 7))
tc

In [ ]:
#Calcular prediccion de la feature para IS_RECID
#Predictions for IS_recid
def modelo2(caso):
    if  caso['is_recid'] == 1 or caso['is_violent_recid'] == 1 and caso['r_offense_date'] > fecha_limite and caso['age_cat'=='Less than 25'] and caso['sex'] == 'Male':
        return 1
    else:
        return 0

In [ ]:
score2 = df1.apply(lambda x: modelo2(x), axis = 1).rename('Predicción')

In [ ]:
roc_auc_score((df1['decile_score'] >= 7), score2)

#Feature que nos entrega un valor de aciertos cercano al 61%. 

In [ ]:
tc1 = pd.crosstab(score2, (df1['decile_score'] >= 7))
tc1

As can be seen in the upper contingency table, type I errors correspond to false positives in the upper right corner.
where 1458 people are considered according to the prediction as not possible
Repeat offenders do but actually have a Decile_Score> = 7.
Type II errors correspond to the lower left corner (2200 cases) where our prediction gives results of possible repeat offenders (> = 7) but in reality they are with a decile_score <7. False negatives.

Como se puede apreciar en la tabla de contigencia superior los errores de tipo I corresponden a los falsos positivos de la esquina superior derecha
en donde 1458 personas se consideran segun la predicción como no posibles 
reincidentes te pero en realidad tienen un Decile_Score >= 7.
Los errores de tipo II corresponden a la esquina inferior izquierda (2200 casos) en donde nuestra predicción entrega resultados de posibles reincidentes (>=7) pero en realidad estan con un decile_score <7. Falsos negativos.

## 4.- Graph the differences according to factors of race, sex and age
## 4.- Graficar las diferencias de acuerdos a factores de raza, sexo y edad 

In [ ]:
#Decile_score and v_decile_score -1 values are removed to plot
#Se eliminan valores -1 de decile_score y v_decile_score para graficar
df1 = df1.drop(df1[df1['v_decile_score']==-1].index)
df1 = df1.drop(df1[df1['decile_score']==-1].index)


In [ ]:

#RACE
dfgb = df1.groupby("race")
race_count = df1.groupby("race")["name"].count()

fig, ax = plt.subplots(3, figsize=(14, 8))

for (i, race) in enumerate(["African-American", "Caucasian", "Hispanic"]):
    (
        (dfgb
            .get_group(race)
            .groupby("decile_score")["name"].count() / race_count[race]
        )
        .plot(kind="bar", ax=ax[i], color="#353535")
    )
    ax[i].set_ylabel(race)
    ax[i].set_xlabel("")
   
    ax[i].set_ylim(0, 0.32)

fig.suptitle("Score Frequency by Race")
plt.show()

#### As can be seen in the graph, the distribution in the decile score of black people is uniformly distributed from 1 to 10, while for the other races, the vast majority of cases are in the lower values of the decile_score.

#### Como se puede ver en el gráfico, la distribucion en el decile score de las personas de raza negra se encuentra uniformemente distribuida del 1 al 10, Mientras que para las demas razas, la gran mayoría de los casos se encuentran en los valores mas bajos del decile_score.

In [ ]:
#Sex
dfgb = df1.groupby("sex")
race_count = df1.groupby("sex")["name"].count()

fig, ax = plt.subplots(2, figsize=(14, 8))

for (i, sex) in enumerate(["Male", "Female"]):
    (
        (dfgb
            .get_group(sex)
            .groupby("decile_score")["name"].count() 
         #/ race_count[race]
        )
        .plot(kind="bar", ax=ax[i], color="#353535")
    )
    ax[i].set_ylabel(sex)
    ax[i].set_xlabel("")
   
 #   ax[i].set_ylim(0, 0.)

fig.suptitle("Score Frequency by Race")
plt.show()

In the case of the differences for men and women, a tendency can be observed in the graph that fewer women are at the top of the decile_score than men.

Para el caso de las diferencias para hombres y mujeres se puede observar en la gráfica una tendencia a que menos mujeres estan en lo mas alto del decile_score a diferencia de los hombres. 

In [ ]:
#Age
dfgb = df1.groupby("age_cat")
race_count = df1.groupby("age_cat")["name"].count()

fig, ax = plt.subplots(3, figsize=(14, 8))

for (i, age_cat) in enumerate(["Less than 25", "25 - 45", "Greater than 45"]):
    (
        (dfgb
            .get_group(age_cat)
            .groupby("decile_score")["name"].count() 
        )
        .plot(kind="bar", ax=ax[i], color="#353535")
    )
    ax[i].set_ylabel(age_cat)
    ax[i].set_xlabel("")
   
 #   ax[i].set_ylim(0, 0.)

fig.suptitle("Score Frequency by age category")
plt.show()

Finally, when analyzing the graphs for age, large differences can be seen between the 3 age groups plotted, where those under 25 years of age have a distribution that is highly oriented towards the highest indices, rather than differences from the older age group.

Finalmente al analisis de los graficos para edad, se pueden ver grandes diferencias entre los 3 grupos etarios graficados en donde los menores de 25 años tienen una distribución muy orientada hacia los indices mas alto a diferencias del grupo de mayor edad. 

5.- ¿Para qué tipo de riesgos, el de delitos generales o el de delitos violentos, tiene el sistema más capacidad predictiva?
5.- For what types of risks, general crimes or violent crimes, does the system have the most predictive capacity?

According to the Feature created, both for is_recid and for is_violent recid the results are close to 54% correct. However, when using both columns of information (is_recid == 1 and is_violent_recid == 1) it is possible to obtain a value close to 60%, so it could be assumed that both data have a similar predictive power ///


De acuerdo a la Feature creada, tanto para is_recid, como para is_violent recid los resultados son cercanos al 54% de acierto. Sin embargo al utilizar ambas columans de informacion (is_recid == 1 and is_violent_recid == 1) se logta obtener un valor cercano al 60% por lo que podria asumir que ambas datas poseen un poder predictivo similar///